In [ ]:
!pip install -U spacy
!pip install spacy_transformers

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spacy.__version__

'3.7.6'

In [ ]:
!nvidia-smi

Sat Aug 31 12:44:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
cv_data=json.load(open('/content/drive/MyDrive/Stage/Extraire/dataset/dataset.json','r'))

In [ ]:
len(cv_data)

1014

In [ ]:
cv_data[0]

['\xa0 \xa0\nContact\nwww.linkedin.com/in/omjagri\n(LinkedIn)\nTop Skills\nphp\nMySQL\nJavaScript\nCertifications\nPhp & Js Om Prakash Jagri\nFull Stack Developer | PHP | Laravel | Vue Js\nKathmandu, Bāgmatī, Nepal\nSummary\nExperienced Developer with a demonstrated history of working in\nthe information technology and services industry. Skilled in Laravel,\nPHP, Cascading Style Sheets (CSS), JavaScript, vue js and MySQL.\nStrong engineering professional with a B.sc.CSIT(Bachelors of\nScience in Computer Science and Information Technology) focused\nin Computer Science from Tribhuvan University, Institute of Science\n& Tchnology. \nExperience\nSearchable Design LLC\nSoftware Developer\nJune 2021\xa0-\xa0Present\xa0 (1 year 7 months)\nNepal\nFull Stack Developer Laravel with Vue Js\nBenekiva\nTechnical Documentation\nSeptember 2021\xa0-\xa0Present\xa0 (1 year 4 months)\nUnited States\nBidhee\n3 years 9 months\nLaravel Developer\nMarch 2018\xa0-\xa0May 2021\xa0 (3 years 3 months)\nBaneswa

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/Stage/Extraire/config/base_config.cfg /content/drive/MyDrive/Stage/Extraire/config/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Stage/ResumeParser/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        ents = []
        entity_indices = []

        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue

            entity_indices = entity_indices + list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = str([start, end]) + "    " + str(text) + "\n"
                file.write(err_data)

            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
          pass

    return db

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.2)

In [ ]:
len(train),len(test)

(811, 203)

In [ ]:
file = open('/content/drive/MyDrive/Stage/Extraire/model/train_file.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/Stage/Extraire/model/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/Stage/Extraire/model/test_data.spacy')

file.close()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 203/203 [00:01<00:00, 119.80it/s]


In [ ]:
!python -m spacy train /content/drive/MyDrive/Stage/Extraire/config/config.cfg --output /content/drive/MyDrive/Stage/Extraire/model/output --paths.train /content/drive/MyDrive/Stage/Extraire/model/train_data.spacy --paths.dev /content/drive/MyDrive/Stage/Extraire/model/test_data.spacy --gpu-id 0

✔ Created output directory:
/content/drive/MyDrive/Stage/ResumeParser/model/output
ℹ Saving to output directory:
/content/drive/MyDrive/Stage/ResumeParser/model/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new downlo

In [ ]:
nlp=spacy.load('/content/drive/MyDrive/Stage/Extraire/model/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pi

In [ ]:
!pip install PyMuPDF

In [ ]:
import sys, fitz

fname = '/content/drive/MyDrive/Stage/BubbleCV.pdf'
doc = fitz.open(fname)

In [ ]:
text = ""
for page in doc:
    text = text + str(page.get_text())

In [ ]:
text

'Ismail Ifakir\nData Scientist\nProfile\nI am a 23-year-old with a master’s degree in Data Science\nand Intelligent Systems.\nPassionate about data science, I\nseek opportunities to apply my expertise in data analysis,\nmachine learning, and AI. I am motivated to contribute to\nexciting projects and continue my professional development.\nProjects\n2024\n↑\n2019\nDynamic Website for a Private School\n• Bachelor’s Degree Completion Project:\nRealization of\na Dynamic Website for a Private School the project\ninvolved the use of technologies such as PHP, HTML.\nCSS and JavaScript\nSpeaker recognition\n• Deep Learning: RNN,LSTM,BLSTM,GRU.\n• NLP\nIntrusion detection systems\n• Machine Learning: Random Forest,SVM,Decision Tree.\n• Realization of a binary classification(Normal or atack).\nImage Classification\n• Deep Learning: CNN.\n• Classification(dog and cat).\n• VGG16,ResNet.\nResearch Laboratory Management Application\n• Spring Boot.\n• web technologies(HTML,CSS and JavaScript).\n• MySQ

In [ ]:
doc=nlp(text)
for ent in doc.ents:
  print(ent.text," >>>>>>>> ", ent.label_)

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Ismail Ifakir  >>>>>>>>  NAME
Faculty of Mohammed First - Nador  >>>>>>>>  COLLEGE NAME
Master’s in Data Sciences and Intelligent Systems
2022  >>>>>>>>  DEGREE
Faculty of Ibn Zohr - Ouarzazate  >>>>>>>>  COLLEGE NAME
Bachelor’s in Mathematics and Computer Science  >>>>>>>>  DEGREE
R  >>>>>>>>  COMPANIES WORKED AT
ismailifakir2@gmail.com  >>>>>>>>  EMAIL ADDRESS
ismail  >>>>>>>>  LOCATION
Flask  >>>>>>>>  SKILLS
Matplotlib  >>>>>>>>  SKILLS
Scikit-learn  >>>>>>>>  SKILLS
TensorFlow  >>>>>>>>  SKILLS
Keras  >>>>>>>>  SKILLS
PyTorch  >>>>>>>>  SKILLS
NLP Specialezation
○
Oracle Cloud Infrastructure 2023 AI  >>>>>>>>  CERTIFICATION
Certified Foundations Associate
○  >>>>>>>>  CERTIFICATION
Transformer
Models
and  >>>>>>>>  CERTIFICATION
BERT  >>>>>>>>  CERTIFICATION
Model  >>>>>>>>  CERTIFICATION
Tamazight  >>>>>>>>  LANGUAGE
Arabic  >>>>>>>>  LANGUAGE
French  >>>>>>>>  LANGUAGE
Intermediate  >>>>>>>>  SKILLS
English  >>>>>>>>  LANGUAGE


### Extraire Entitie

In [ ]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/Stage/Extraire/dataset/pdf_text.csv')

In [ ]:
df=df.head(500)
df

,Text
0,"\n \nMd. Anis \n 23/H/1 Palm Avenue, Kolka..."
1,PUBLIC RELATIONS SPECIALIST\nhttps://chelseape...
2,CITY CARRIER ASSISTANT\nProfessional Summary\n...
3,TEACHER\nSummary\nA dedicated Teacher and Mari...
4,CONSULTANT\nSummary\nAccomplished Senior Busin...
...,...
495,ROHITH KANNAN A...
496,CURRICULUM VITAE \nEr. BA...
497,\n ...
498,RESUME\n Jitender kumar\n pbkuldip@gmail.com...


In [ ]:
import pandas as pd
import spacy


# Function to extract all entities from a single text entry
def extract_entities(text, nlp):
    if isinstance(text, str):  # Ensure the text is a string
        doc = nlp(text)
        # Collect all entities with their labels
        entities = [f"{ent.text} ({ent.label_})" for ent in doc.ents]
        return ", ".join(entities)  # Join the list of entities into a single string
    return ''

# Apply the function to each row in the 'Text' column and store the result in a new column 'Entities'
df['Entities'] = df['Text'].apply(lambda x: extract_entities(x, nlp))

# Display the DataFrame with the new 'Entities' column
print(df[['Text', 'Entities']])

In [ ]:
output_path = '/content/drive/MyDrive/Stage/Extraire/dataset/extract_entities.csv'
df.to_csv(output_path, index=False)

print(f"Data saved to {output_path}")

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Stage/Extraire/dataset/extract_entities.csv')
df1

,Text,Entities
0,"\n \nMd. Anis \n 23/H/1 Palm Avenue, Kolka...","ani.elc@gmail.com (EMAIL ADDRESS), Electrician..."
1,PUBLIC RELATIONS SPECIALIST\nhttps://chelseape...,"PUBLIC RELATIONS SPECIALIST (SKILLS), https://..."
2,CITY CARRIER ASSISTANT\nProfessional Summary\n...,"Morgan State University (UNIVERSITY), Bachelor..."
3,TEACHER\nSummary\nA dedicated Teacher and Mari...,"TEACHER (LINKEDIN LINK), Robert Morris Univers..."
4,CONSULTANT\nSummary\nAccomplished Senior Busin...,"CONSULTANT (LINKEDIN LINK), International Inst..."
...,...,...
495,ROHITH KANNAN A...,"SCHOLASTICS (COMPANIES WORKED AT), Bachelor\n ..."
496,CURRICULUM VITAE \nEr. BA...,"yadavppj25@gmail.com (EMAIL ADDRESS), Institut..."
497,\n ...,"Masters, IT(Information Technology) Administra..."
498,RESUME\n Jitender kumar\n pbkuldip@gmail.com...,"Jitender kumar (NAME), pbkuldip@gmail.com (EMA..."


### Analyse Similarity

In [ ]:
!pip install sentence_transformers

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load your SBERT model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load your datasets
entite_df = pd.read_csv('/content/drive/MyDrive/Stage/Extraire/dataset/extract_entities.csv')
job_df = pd.read_csv('/content/drive/MyDrive/Stage/Extraire/dataset/df_Dublin.csv', encoding='ISO-8859-1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
entite_df

,Text,Entities
0,"\n \nMd. Anis \n 23/H/1 Palm Avenue, Kolka...","ani.elc@gmail.com (EMAIL ADDRESS), Electrician..."
1,PUBLIC RELATIONS SPECIALIST\nhttps://chelseape...,"PUBLIC RELATIONS SPECIALIST (SKILLS), https://..."
2,CITY CARRIER ASSISTANT\nProfessional Summary\n...,"Morgan State University (UNIVERSITY), Bachelor..."
3,TEACHER\nSummary\nA dedicated Teacher and Mari...,"TEACHER (LINKEDIN LINK), Robert Morris Univers..."
4,CONSULTANT\nSummary\nAccomplished Senior Busin...,"CONSULTANT (LINKEDIN LINK), International Inst..."
...,...,...
495,ROHITH KANNAN A...,"SCHOLASTICS (COMPANIES WORKED AT), Bachelor\n ..."
496,CURRICULUM VITAE \nEr. BA...,"yadavppj25@gmail.com (EMAIL ADDRESS), Institut..."
497,\n ...,"Masters, IT(Information Technology) Administra..."
498,RESUME\n Jitender kumar\n pbkuldip@gmail.com...,"Jitender kumar (NAME), pbkuldip@gmail.com (EMA..."


In [ ]:
job_df

,Unnamed: 0,job_title,job_location,company_name,job_description,city
0,1,Financial Data Sampling Analyst,Dublin,Accenture,The focus of this role is on data review and c...,Dublin
1,2,Research Analyst,Dublin,HokuApps Pte Ltd,Analytical and data driven roles in leading gl...,Dublin
2,3,Data Analyst II - Enterprise Data,Carlow,Unum,"Mastery of data profiling, including data anom...",Dublin
3,4,International Search Quality Operations - Analyst,Dublin,Indeed,Knowledge of the UK and Ireland market to eval...,Dublin
4,5,Data Analyst,Dublin,Next Generation Recruitment,You have experience as a data analyst in a cli...,Dublin
...,...,...,...,...,...,...
598,599,Associate Manager Raw Materials,Limerick,CareerWise Recruitment,"By applying for this position, you are consent...",Dublin
599,600,Reseller Operations Supply Chain Account Manag...,Cork,Apple,"Prepare partner demand data, and conduct detai...",Dublin
600,601,Associate Manager Chemistry,Limerick,CareerWise Recruitment,"By applying for this position, you are consent...",Dublin
601,602,Data Analyst - Utilities company,County Dublin,EXECUTIVE TALENT,"Knowledge of data modelling, data cleansing, a...",Dublin


In [ ]:
# Encode the sentences in 'entite' and 'job'
entite_embeddings = model.encode(entite_df['Entities'].tolist(), convert_to_tensor=True)
job_embeddings = model.encode(job_df['job_description'].tolist(), convert_to_tensor=True)

In [ ]:
# Calculate cosine similarities between entite and job embeddings
similarities = util.pytorch_cos_sim(entite_embeddings, job_embeddings)

In [ ]:
results = []

# Iterate over each 'entite'
for i in range(len(entite_df)):
    # Find the job with the maximum similarity for this 'entite'
    max_similarity_idx = similarities[i].argmax().item()
    max_similarity_score = similarities[i][max_similarity_idx].item()
    best_job = job_df['job_description'].iloc[max_similarity_idx]

    # Store the result
    results.append({
        'Entities': entite_df['Entities'].iloc[i],
        'job_description': best_job,
        'similarity': max_similarity_score
    })

# Convert the list of results into a DataFrame
similarity_df = pd.DataFrame(results)

# Display the first few rows of the result
print(similarity_df.head())

                                            Entities  \
0  ani.elc@gmail.com (EMAIL ADDRESS), Electrician...   
1  PUBLIC RELATIONS SPECIALIST (SKILLS), https://...   
2  Morgan State University (UNIVERSITY), Bachelor...   
3  TEACHER (LINKEDIN LINK), Robert Morris Univers...   
4  CONSULTANT (LINKEDIN LINK), International Inst...   

                                     job_description  similarity  
0  Be a subject matter expert in your area of rec...    0.398546  
1  Likely job titles/previous roles would be comm...    0.489693  
2  BA/BS degree in Computer Science or Business, ...    0.414785  
3  Use data and internal reviews to make recommen...    0.355625  
4  2+ years experience working in a data analytic...    0.608316  


In [ ]:
similarity_df.to_csv('/content/drive/MyDrive/Stage/Similarity/similarity_results.csv', index=False)

In [ ]:
df3=pd.read_csv('/content/drive/MyDrive/Stage/Similarity/similarity_results.csv')
df3

,Entities,job_description,similarity
0,"ani.elc@gmail.com (EMAIL ADDRESS), Electrician...",Be a subject matter expert in your area of rec...,0.398546
1,"PUBLIC RELATIONS SPECIALIST (SKILLS), https://...",Likely job titles/previous roles would be comm...,0.489693
2,"Morgan State University (UNIVERSITY), Bachelor...","BA/BS degree in Computer Science or Business, ...",0.414785
3,"TEACHER (LINKEDIN LINK), Robert Morris Univers...",Use data and internal reviews to make recommen...,0.355625
4,"CONSULTANT (LINKEDIN LINK), International Inst...",2+ years experience working in a data analytic...,0.608316
...,...,...,...
495,"SCHOLASTICS (COMPANIES WORKED AT), Bachelor\n ...",Likely job titles/previous roles would be comm...,0.460834
496,"yadavppj25@gmail.com (EMAIL ADDRESS), Institut...",Be a subject matter expert in your area of rec...,0.410086
497,"Masters, IT(Information Technology) Administra...","Advanced IT, data management and modeling skil...",0.558351
498,"Jitender kumar (NAME), pbkuldip@gmail.com (EMA...",Be a subject matter expert in your area of rec...,0.359258
